In [ ]:
import tensorflow as tf
import kagglehub
import os
import cv2
import numpy as np

#### data processing

In [ ]:
!pip install tensorflow kagglehub opencv-python matplotlib numpy





#path = kagglehub.dataset_download("emmarex/plantdisease")
#print("Dataset path:", path)

train = "/kaggle/input/plant-disease-dataset/plant_disease_dataset/train"
val = "/kaggle/input/plant-disease-dataset/plant_disease_dataset/val"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)






In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.25),
    tf.keras.layers.RandomZoom(0.25),
    tf.keras.layers.RandomContrast(0.3),
    tf.keras.layers.RandomBrightness(0.3),
])


In [ ]:
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Classes:", NUM_CLASSES)

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.Lambda(preprocess_input),
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])
'''model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)'''
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)



In [ ]:
import cv2
import numpy as np

def predict_image_keras(model, image_path, class_names):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)      # NO /255
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img)
    class_id = np.argmax(preds[0])
    confidence = preds[0][class_id]

    return class_names[class_id], confidence


In [ ]:
label, conf = predict_image_keras(
    model,
    "/kaggle/input/plant-disease-dataset/plant_disease_dataset/test/Bacterial spot (Peach)/Bacterial spot (Peach)_1.JPG",
    class_names
)

print(label, conf)


### loading the model after training

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("crop_disease_model_processed.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved for mobile deployment")


In [ ]:
for layer in base_model.layers[-20:]:
    print(layer.name)


In [ ]:
print(input_details)


In [ ]:
interpreter = tf.lite.Interpreter(model_path=model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
import cv2

In [ ]:
def predict_image_tflite(interpreter, image_path, class_names):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)   # ⚠️ NO /255 because preprocess_input is inside model
    img = np.expand_dims(img, axis=0)

    interpreter.set_tensor(input_details[0]["index"], img)
    interpreter.invoke()

    preds = interpreter.get_tensor(output_details[0]["index"])

    class_id = np.argmax(preds)
    confidence = np.max(preds)

    return class_names[class_id], confidence


In [ ]:
print(class_names)

In [ ]:
label, conf = predict_image_tflite(
    interpreter,
    "/kaggle/input/plant-disease-dataset/plant_disease_dataset/test/Bacterial Blight (Cotton)/Bacterial Blight (Cotton)_1.jpg",
    class_names
)

print(label, conf)
